![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

Subset, clean, and reformat the bank_marketing.csv dataset to create and store three new files based on the requirements detailed in the notebook.  
* Split and tidy bank_marketing.csv, storing as three DataFrames called client, campaign, and economics, each containing the columns outlined in the notebook and formatted to the data types listed.
* Save the three DataFrames to csv files, without an index, as client.csv, campaign.csv, and economics.csv respectively.

In [16]:
import pandas as pd
import numpy as np

# Start coding here...

In [17]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [18]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [20]:
# Replacing '.' with '_' for 'job' column.
df['job'] = df['job'].str.replace('.','_', regex = False).astype(object)
df['job'].unique()

array(['housemaid', 'services', 'admin_', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [21]:
# Replacing '.' and 'unknown' with '_' and np.NaN, respectively, for 'education' column..
df['education'] = (df['education'].str.replace('.', '_', regex = False)
                   .replace('unknown', np.NaN)
                   .astype(object)
                   )
df['education'].unique()

array(['basic_4y', 'high_school', 'basic_6y', 'basic_9y',
       'professional_course', nan, 'university_degree', 'illiterate'],
      dtype=object)

In [22]:
# Defining columns need to be transformed as boolean (yes = 1, otherwise 0).
df_bool = ['credit_default', 'mortgage', 'campaign_outcome']

for col in df_bool:
    df[col] = (df[col].apply(lambda x: 1 
                             if (x == 'yes') or (x == 1) # x == 1, so it won't overwrite once yes are transformed.
                             else 0)
              .astype(bool))

bool_values = []
for col in df_bool:
    bool_values.append(df[col].value_counts(dropna = False))

distinct_bool = pd.DataFrame(bool_values)
distinct_bool

,False,True
credit_default,41185,3
mortgage,19612,21576
campaign_outcome,36548,4640


In [23]:
# Boolean encoding the 'previous_outcome' column: success = 1, otherwise 0.
df['previous_outcome'] = (df['previous_outcome'].apply(lambda x: 1 
                                    if (x == 'success') or (x == 1) # x == 1, so it won't overwrite once yes are transformed.
                                    else 0) 
                         .astype(bool)) 

df['campaign_outcome'].unique()

array([False,  True])

In [24]:
df['month'].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'mar', 'apr',
       'sep'], dtype=object)

In [25]:
# Transforming worded month into their respective month digit (e.g. jan = 1, feb = 2, etc.).
df['month'] = pd.to_datetime(df['month'], format='%b').dt.month
df['month'].unique()

array([ 5,  6,  7,  8, 10, 11, 12,  3,  4,  9])

In [26]:
# Defining 'year' column with values 2022.
df['year'] = 2022
# Combining 'year', 'month', and 'day' into a single datetime column to achieve yyyy-mm-dd format.
df['last_contact_date'] = pd.to_datetime(df[['year', 'month', 'day']])
# Converts the datetime column into strings with the format "YYYY-MM-DD".
df['last_contact_date'] = df['last_contact_date'].dt.strftime('%Y-%m-%d')
# Tranforming the dtype into datetime64[ns].
df['last_contact_date'] = pd.to_datetime(df['last_contact_date'])
df['last_contact_date'].unique()

array(['2022-05-13T00:00:00.000000000', '2022-05-19T00:00:00.000000000',
       '2022-05-23T00:00:00.000000000', '2022-05-27T00:00:00.000000000',
       '2022-05-03T00:00:00.000000000', '2022-05-05T00:00:00.000000000',
       '2022-05-12T00:00:00.000000000', '2022-05-21T00:00:00.000000000',
       '2022-05-08T00:00:00.000000000', '2022-05-09T00:00:00.000000000',
       '2022-05-29T00:00:00.000000000', '2022-05-14T00:00:00.000000000',
       '2022-05-01T00:00:00.000000000', '2022-05-06T00:00:00.000000000',
       '2022-05-02T00:00:00.000000000', '2022-05-16T00:00:00.000000000',
       '2022-05-20T00:00:00.000000000', '2022-05-10T00:00:00.000000000',
       '2022-05-28T00:00:00.000000000', '2022-05-30T00:00:00.000000000',
       '2022-05-22T00:00:00.000000000', '2022-05-25T00:00:00.000000000',
       '2022-05-11T00:00:00.000000000', '2022-05-17T00:00:00.000000000',
       '2022-05-15T00:00:00.000000000', '2022-05-26T00:00:00.000000000',
       '2022-05-18T00:00:00.000000000', '2022-05-04

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   age                         41188 non-null  int64         
 2   job                         41188 non-null  object        
 3   marital                     41188 non-null  object        
 4   education                   39457 non-null  object        
 5   credit_default              41188 non-null  bool          
 6   mortgage                    41188 non-null  bool          
 7   month                       41188 non-null  int64         
 8   day                         41188 non-null  int64         
 9   contact_duration            41188 non-null  int64         
 10  number_contacts             41188 non-null  int64         
 11  previous_campaign_contacts  41188 non-null  int64     

In [28]:
# Defining the client dataframe columns from df.
client = df[['client_id', 'age', 'job', 'marital', 
             'education', 'credit_default', 'mortgage']]
# Saving the client dataframe as csv, ignoring its indices.
client.to_csv('client.csv', index = False)
print('client.csv saved successfully')

client.csv saved successfully


In [29]:
# Defining the campaign dataframe columns from df.
campaign = df[['client_id', 'number_contacts', 'contact_duration',
               'previous_campaign_contacts', 'previous_outcome', 
               'campaign_outcome', 'last_contact_date']]
# Saving the campaign dataframe as csv, ignoring its indices.
campaign.to_csv('campaign.csv', index = False)
print('campaign.csv saved successfully')

campaign.csv saved successfully


In [30]:
# Defining the economics dataframe columns from df.
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
# Saving the economics dataframe as csv, ignoring its indices.
economics.to_csv('economics.csv', index = False)
print('economics.csv saved successfully')

economics.csv saved successfully
